# Jeopardy Web Scraper

This program scrapes the website: https://j-archive.com to get Jeopardy questions to be used as you wish.

In [73]:
import requests
import time
import json

from bs4 import BeautifulSoup

base_url = "https://j-archive.com/"

In [70]:
# GET EVERY SEASON
list_seasons_URL = base_url + "listseasons.php"
list_seasons_page = requests.get(list_seasons_URL)
soup = BeautifulSoup(list_seasons_page.content, "html.parser")

seasons_json_array = []
seasons_trs = soup.find_all("tr")
for season_tr in seasons_trs:
    # init new episode json object
    season_json = {"name": None, "from_date": None, "to_date": None, "URL": None, "episodes": []}

    #print(season_tr)

    # get 'a' tag for ID, air date, and URL
    a_tag = season_tr.find("a")
    #print(a_tag)
    #season_json["id"] = a_tag.string.split(",")[0].lstrip("#")
    #season_json["air_date"] = a_tag.string[-10:]
    season_json["URL"] = base_url + a_tag["href"]
    season_json["name"] = a_tag.string

    # get 'td' tag for episode name
    td = season_tr.find_all('td')[1]
    string = td.string.lstrip().rstrip()
    if string.find(" to ") > 0:
        dates = string.split(" to ")
        #print(dates)
        season_json["from_date"] = dates[0]
        season_json["to_date"] = dates[1]

    # append episode json to array
    seasons_json_array.append(season_json)

print(seasons_json_array)

[{'name': 'Season 41', 'from_date': '2024-09-09', 'to_date': '2025-07-25', 'URL': 'https://j-archive.com/showseason.php?season=41', 'episodes': []}, {'name': 'Audio-only games', 'from_date': '2024-01-12', 'to_date': '2024-02-02', 'URL': 'https://j-archive.com/showseason.php?season=cwcpi', 'episodes': []}, {'name': 'Season 40', 'from_date': '2023-09-11', 'to_date': '2024-07-26', 'URL': 'https://j-archive.com/showseason.php?season=40', 'episodes': []}, {'name': 'Jeopardy! Masters', 'from_date': '2023-05-08', 'to_date': '2023-05-24', 'URL': 'https://j-archive.com/showseason.php?season=jm', 'episodes': []}, {'name': None, 'from_date': '2022-09-25', 'to_date': '2025-04-23', 'URL': 'https://j-archive.com/showseason.php?season=pcj', 'episodes': []}, {'name': 'Season 39', 'from_date': '2022-09-12', 'to_date': '2023-07-28', 'URL': 'https://j-archive.com/showseason.php?season=39', 'episodes': []}, {'name': 'Jeopardy! National College Championship', 'from_date': '2022-02-08', 'to_date': '2022-02-

In [71]:
# GET A SEASON'S EPISODES

for season in seasons_json_array:

    season_episodes_URL = season["URL"]
    season_episodes_page = requests.get(season_episodes_URL)
    soup = BeautifulSoup(season_episodes_page.content, "html.parser")

    episodes_json_array = []
    episodes = soup.find_all("tr")
    for episode in episodes:
        # init new episode json object
        episode_json = {"id": None, "name": None, "air_date": None, "URL": None}

        # get 'a' tag for ID, air date, and URL
        a_tag = episode.find("a")
        if a_tag.string is not None:
            episode_json["id"] = a_tag.string.split(",")[0].lstrip("#")
            episode_json["air_date"] = a_tag.string[-10:]
        episode_json["URL"] = base_url + a_tag["href"]

        # get 'td' tag for episode name
        td = episode.find_all('td')[1]
        string = td.string.lstrip().rstrip()
        episode_json["name"] = string

        # append episode json to array
        episodes_json_array.append(episode_json)
    
    # wait one second as to not get blocked from the website
    time.sleep(1)

    season["episodes"] = episodes_json_array

In [74]:
print(json.dumps(seasons_json_array, indent=4))

[
    {
        "name": "Season 41",
        "from_date": "2024-09-09",
        "to_date": "2025-07-25",
        "URL": "https://j-archive.com/showseason.php?season=41",
        "episodes": [
            {
                "id": "9290",
                "name": "Jack Goldfisher vs. Alex DeFrank vs. Jamie Hare",
                "air_date": "2025-03-14",
                "URL": "https://j-archive.com/showgame.php?game_id=9143"
            },
            {
                "id": "9289",
                "name": "James Corson vs. Jack Goldfisher vs. Liz Engelhardt",
                "air_date": "2025-03-13",
                "URL": "https://j-archive.com/showgame.php?game_id=9142"
            },
            {
                "id": "9288",
                "name": "James Corson vs. Kristen VanBlargan vs. Brian Nieves",
                "air_date": "2025-03-12",
                "URL": "https://j-archive.com/showgame.php?game_id=9141"
            },
            {
                "id": "9287",
        

In [ ]:
# GET AN EPISODE'S CLUES / ANSWERS

episodes_to_grab_for_test = 1
count = 0
for season in seasons_json_array:
    for episode in season["episodes"]:
        # clear jeopardy cards
        jeopardy_round = []

        # request jeopardy game page
        page = requests.get(episode["URL"])
        soup = BeautifulSoup(page.content, "html.parser")

        # get all jeopardy cards
        #print(soup)

        jeopardy_round_html = soup.find("div", id="jeopardy_round")
        #print(jeopardy_round_html)

        categories = jeopardy_round_html.find_all("td", class_="category_name")
        for category in categories:
            category_column = {"category_name": category.string, "cards": []}
            cards = []

            #card_rows = jeopardy_round_html.find("tbody")
            #find_all("tr", {}, False)
            #print(card_rows[1:])


            jeopardy_round.append(category_column)


        #print(jeopardy_round)
        #print(categories)
        #for 


        count += 1
        if count >= episodes_to_grab_for_test:
            break

    if count >= episodes_to_grab_for_test:
        break


TypeError: 'NoneType' object is not subscriptable